In [1]:
import Pkg
Pkg.activate("..")

  Activating project at `~/Projects/latentplan.jl`


In [2]:
using Test
using PyCall
using Knet
using Debugger: @enter, @bp, @run
using CUDA

if CUDA.functional()
	atype=KnetArray{Float32}
else	
	atype=Array{Float32}
end
cputype=Array{Float32}

Array{Float32}

In [3]:
@pyimport torch
@pyimport numpy

weights = torch.load("test/files/gpt_trained.pt", map_location=torch.device("cpu"))
prior_weights = torch.load("test/files/prior_model.pt", map_location=torch.device("cpu"))

Dict{Any, Any} with 75 entries:
  "blocks.2.ln2.bias"          => PyObject tensor([0., 0., 0., 0., 0., 0., 0., …
  "blocks.0.attn.value.bias"   => PyObject tensor([0., 0., 0., 0., 0., 0., 0., …
  "state_emb.bias"             => PyObject tensor([0., 0., 0., 0., 0., 0., 0., …
  "blocks.1.attn.proj.bias"    => PyObject tensor([0., 0., 0., 0., 0., 0., 0., …
  "blocks.3.attn.value.bias"   => PyObject tensor([0., 0., 0., 0., 0., 0., 0., …
  "blocks.0.ln2.bias"          => PyObject tensor([0., 0., 0., 0., 0., 0., 0., …
  "blocks.0.mlp.0.bias"        => PyObject tensor([0., 0., 0.,  ..., 0., 0., 0.…
  "blocks.3.ln1.weight"        => PyObject tensor([1., 1., 1., 1., 1., 1., 1., …
  "pos_emb"                    => PyObject tensor([[[0., 0., 0.,  ..., 0., 0., …
  "blocks.0.mlp.0.weight"      => PyObject tensor([[ 0.0037,  0.0118, -0.0088, …
  "blocks.2.attn.query.bias"   => PyObject tensor([0., 0., 0., 0., 0., 0., 0., …
  "blocks.3.attn.mask"         => PyObject tensor([[[[1., 0., 0., 0., 0., 0.,

In [17]:
for key in keys(prior_weights)
    println(key) 
end

blocks.2.ln2.bias
blocks.0.attn.value.bias
state_emb.bias
blocks.1.attn.proj.bias
blocks.3.attn.value.bias
blocks.0.ln2.bias
blocks.0.mlp.0.bias
blocks.3.ln1.weight
pos_emb
blocks.0.mlp.0.weight
blocks.2.attn.query.bias
blocks.3.attn.mask
blocks.2.mlp.0.weight
blocks.1.mlp.0.weight
blocks.3.mlp.2.weight
blocks.1.attn.value.weight
blocks.0.attn.query.weight
blocks.2.attn.query.weight
blocks.2.attn.key.weight
blocks.0.attn.mask
blocks.1.ln2.bias
blocks.3.ln2.bias
blocks.2.mlp.0.bias
blocks.1.mlp.2.weight
head.weight
blocks.2.ln2.weight
blocks.2.attn.key.bias
state_emb.weight
blocks.3.ln2.weight
blocks.1.attn.value.bias
blocks.2.mlp.2.weight
blocks.0.mlp.2.bias
blocks.0.ln1.weight
blocks.2.attn.proj.bias
blocks.3.mlp.0.bias
blocks.3.attn.key.weight
blocks.1.attn.query.weight
blocks.0.attn.value.weight
blocks.1.ln2.weight
blocks.1.attn.mask
blocks.1.attn.query.bias
blocks.0.ln1.bias
blocks.3.attn.query.weight
blocks.3.ln1.bias
blocks.3.attn.proj.bias
blocks.0.attn.query.bias
blocks.0.attn.

In [4]:
include("datasets/sequence.jl")
include("models/common.jl")
include("models/transformers.jl")
include("models/vqvae.jl")
include("setup.jl")

No module named 'flow'
No module named 'carla'
pybullet build time: Feb  2 2023 19:56:04


read_config (generic function with 1 method)

In [5]:
super_args = Dict{String, Any}(
    "dataset"=> "hopper-medium-replay-v2",
    "exp_name"=> "debug",
    "seed"=> 42,
    "config"=> "../config/vqvae.jl",
    "representation_path" => "", #TODO
)

args = parser(super_args, experiment="plan")

args["logbase"] = expanduser(args["logbase"])
args["savepath"] = expanduser(args["savepath"])

env_name = occursin("-v", args["dataset"]) ? args["dataset"] : args["dataset"] * "-v0"

[ utils/setup ] Reading config: ../config/vqvae.jl:hopper_medium_replay_v2
/Users/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/debug/0 already exists. Proceeding...
Made directory /Users/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/debug/0


"hopper-medium-replay-v2"

In [6]:
dataset_config = Knet.load(joinpath("/Users/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42", "dataset_config.jld2"), "config")

dataset = SequenceDataset(
    dataset_config["env_name"];
    penalty=dataset_config["penalty"],
    sequence_length=dataset_config["sequence_length"], 
    step=dataset_config["step"], 
    discount=dataset_config["discount"], 
    disable_goal=dataset_config["disable_goal"], 
    normalize_raw=dataset_config["normalize_raw"], 
    normalize_reward=dataset_config["normalize_reward"],
    max_path_length=dataset_config["max_path_length"],
    atype=dataset_config["atype"]
)

obs_dim = dataset.observation_dim
act_dim = dataset.action_dim
transition_dim = dataset.joined_dim+1

[ datasets/sequence ] Sequence length: 25 | Step: 1 | Max path length: 1000
[ datasets/sequence ] Loading...


/Users/mehmeteneserciyes/.julia/conda/3/lib/python3.10/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


✓
[ datasets/sequence ] Segmenting...

load datafile: 100%|███████████████████████████| 11/11 [00:00<00:00, 14.22it/s]



✓


Calculating values 100%|█████████████████████████████████| Time: 0:00:04


17

# Representation model init and weight loading

In [7]:
model_config = Knet.load(joinpath("/Users/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42", "model_config.jld2"), "config")

OrderedCollections.OrderedDict{String, Any} with 52 entries:
  "model"                      => "VQTransformer"
  "tag"                        => "experiment"
  "state_conditional"          => true
  "N"                          => 100
  "discount"                   => 0.99
  "n_layer"                    => 4
  "n_head"                     => 4
  "n_epochs_ref"               => 50
  "n_saves"                    => 3
  "logbase"                    => "~/logs_julia/"
  "device"                     => "cuda"
  "K"                          => 512
  "latent_step"                => 3
  "n_embd"                     => 512
  "trajectory_embd"            => 512
  "batch_size"                 => 512
  "learning_rate"              => 0.0002
  "lr_decay"                   => false
  "seed"                       => 42
  "embd_pdrop"                 => 0.0
  "resid_pdrop"                => 0.0
  "attn_pdrop"                 => 0.0
  "step"                       => 1
  "subsampled_sequence_length" => 

In [8]:
representation = VQContinuousVAE(model_config);

In [9]:
# encoder
representation.model.embed.w = Param(atype(weights["model.embed.weight"][:cpu]()[:numpy]()))
representation.model.embed.b = Param(atype(weights["model.embed.bias"][:cpu]()[:numpy]()))

representation.model.pos_emb = Param(atype(permutedims(weights["model.pos_emb"][:cpu]()[:numpy](), (3,2,1))))

for i in 1:model_config["n_layer"]
    representation.model.encoder.layers[i].ln1.a = Param(atype(weights["model.encoder.$(i-1).ln1.weight"][:cpu]()[:numpy]()))
    representation.model.encoder.layers[i].ln1.b = Param(atype(weights["model.encoder.$(i-1).ln1.bias"][:cpu]()[:numpy]()))
    representation.model.encoder.layers[i].ln2.a = Param(atype(weights["model.encoder.$(i-1).ln2.weight"][:cpu]()[:numpy]()))
    representation.model.encoder.layers[i].ln2.b = Param(atype(weights["model.encoder.$(i-1).ln2.bias"][:cpu]()[:numpy]()))

    representation.model.encoder.layers[i].attn.key.w = Param(atype(weights["model.encoder.$(i-1).attn.key.weight"][:cpu]()[:numpy]()))
    representation.model.encoder.layers[i].attn.key.b = Param(atype(weights["model.encoder.$(i-1).attn.key.bias"][:cpu]()[:numpy]()))
    representation.model.encoder.layers[i].attn.query.w = Param(atype(weights["model.encoder.$(i-1).attn.query.weight"][:cpu]()[:numpy]()))
    representation.model.encoder.layers[i].attn.query.b = Param(atype(weights["model.encoder.$(i-1).attn.query.bias"][:cpu]()[:numpy]()))
    representation.model.encoder.layers[i].attn.value.w = Param(atype(weights["model.encoder.$(i-1).attn.value.weight"][:cpu]()[:numpy]()))
    representation.model.encoder.layers[i].attn.value.b = Param(atype(weights["model.encoder.$(i-1).attn.value.bias"][:cpu]()[:numpy]()))
    representation.model.encoder.layers[i].attn.proj.w = Param(atype(weights["model.encoder.$(i-1).attn.proj.weight"][:cpu]()[:numpy]()))
    representation.model.encoder.layers[i].attn.proj.b = Param(atype(weights["model.encoder.$(i-1).attn.proj.bias"][:cpu]()[:numpy]()))

    representation.model.encoder.layers[i].mlp.layers[1].w = Param(atype(weights["model.encoder.$(i-1).mlp.0.weight"][:cpu]()[:numpy]()))
    representation.model.encoder.layers[i].mlp.layers[1].b = Param(atype(weights["model.encoder.$(i-1).mlp.0.bias"][:cpu]()[:numpy]()))
    representation.model.encoder.layers[i].mlp.layers[3].w = Param(atype(weights["model.encoder.$(i-1).mlp.2.weight"][:cpu]()[:numpy]()))
    representation.model.encoder.layers[i].mlp.layers[3].b = Param(atype(weights["model.encoder.$(i-1).mlp.2.bias"][:cpu]()[:numpy]()))
end

representation.model.cast_embed.w = Param(atype(weights["model.cast_embed.weight"][:cpu]()[:numpy]()))
representation.model.cast_embed.b = Param(atype(weights["model.cast_embed.bias"][:cpu]()[:numpy]()))

# Decoder
representation.model.latent_mixing.w = Param(atype(weights["model.latent_mixing.weight"][:cpu]()[:numpy]()))
representation.model.latent_mixing.b = Param(atype(weights["model.latent_mixing.bias"][:cpu]()[:numpy]()))

for i in 1:model_config["n_layer"]
    representation.model.decoder.layers[i].ln1.a = Param(atype(weights["model.decoder.$(i-1).ln1.weight"][:cpu]()[:numpy]()))
    representation.model.decoder.layers[i].ln1.b = Param(atype(weights["model.decoder.$(i-1).ln1.bias"][:cpu]()[:numpy]()))
    representation.model.decoder.layers[i].ln2.a = Param(atype(weights["model.decoder.$(i-1).ln2.weight"][:cpu]()[:numpy]()))
    representation.model.decoder.layers[i].ln2.b = Param(atype(weights["model.decoder.$(i-1).ln2.bias"][:cpu]()[:numpy]()))

    representation.model.decoder.layers[i].attn.key.w = Param(atype(weights["model.decoder.$(i-1).attn.key.weight"][:cpu]()[:numpy]()))
    representation.model.decoder.layers[i].attn.key.b = Param(atype(weights["model.decoder.$(i-1).attn.key.bias"][:cpu]()[:numpy]()))
    representation.model.decoder.layers[i].attn.query.w = Param(atype(weights["model.decoder.$(i-1).attn.query.weight"][:cpu]()[:numpy]()))
    representation.model.decoder.layers[i].attn.query.b = Param(atype(weights["model.decoder.$(i-1).attn.query.bias"][:cpu]()[:numpy]()))
    representation.model.decoder.layers[i].attn.value.w = Param(atype(weights["model.decoder.$(i-1).attn.value.weight"][:cpu]()[:numpy]()))
    representation.model.decoder.layers[i].attn.value.b = Param(atype(weights["model.decoder.$(i-1).attn.value.bias"][:cpu]()[:numpy]()))
    representation.model.decoder.layers[i].attn.proj.w = Param(atype(weights["model.decoder.$(i-1).attn.proj.weight"][:cpu]()[:numpy]()))
    representation.model.decoder.layers[i].attn.proj.b = Param(atype(weights["model.decoder.$(i-1).attn.proj.bias"][:cpu]()[:numpy]()))

    representation.model.decoder.layers[i].mlp.layers[1].w = Param(atype(weights["model.decoder.$(i-1).mlp.0.weight"][:cpu]()[:numpy]()))
    representation.model.decoder.layers[i].mlp.layers[1].b = Param(atype(weights["model.decoder.$(i-1).mlp.0.bias"][:cpu]()[:numpy]()))
    representation.model.decoder.layers[i].mlp.layers[3].w = Param(atype(weights["model.decoder.$(i-1).mlp.2.weight"][:cpu]()[:numpy]()))
    representation.model.decoder.layers[i].mlp.layers[3].b = Param(atype(weights["model.decoder.$(i-1).mlp.2.bias"][:cpu]()[:numpy]()))
end

representation.model.ln_f.a = Param(atype(weights["model.ln_f.weight"][:cpu]()[:numpy]()))
representation.model.ln_f.b = Param(atype(weights["model.ln_f.bias"][:cpu]()[:numpy]()))

representation.model.predict.w = Param(atype(weights["model.predict.weight"][:cpu]()[:numpy]()))
representation.model.predict.b = Param(atype(weights["model.predict.bias"][:cpu]()[:numpy]()))

# codebook
representation.model.codebook.embedding = Param(atype(weights["model.codebook.embedding"][:cpu]()[:numpy]()'))
representation.model.codebook.ema_count = Param(atype(weights["model.codebook.ema_count"][:cpu]()[:numpy]()))
representation.model.codebook.ema_w = Param(atype(weights["model.codebook.ema_w"][:cpu]()[:numpy]()'))

# padding vector
representation.padding_vector = atype(normalize_joined_single(dataset, atype(zeros(representation.transition_dim-1))));

# TransformerPrior Init and Model Loading

In [10]:
args = parser(super_args, experiment="train")
args["logbase"] = expanduser(args["logbase"])
args["savepath"] = expanduser(args["savepath"])
block_size = args["subsampled_sequence_length"] ÷ args["latent_step"]
obs_dim = dataset.observation_dim

[ utils/setup ] Reading config: ../config/vqvae.jl:hopper_medium_replay_v2
/Users/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/debug/ already exists. Proceeding...
Made directory /Users/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/debug/


11

In [11]:
model_config = deepcopy(args)
model_config["block_size"] = block_size
model_config["observation_dim"] = obs_dim
model_config["n_embd"] = args["n_embd"] * args["n_head"]

512

In [13]:
# turn off dropout
model_config["embd_pdrop"] = 0.0f0

0.0f0

In [14]:
model = TransformerPrior(model_config);

In [ ]:
# encoder
model.tok_emb = Param(atype(prior_weights["tok_emb.weight"][:cpu]()[:numpy]()))
model.pos_emb = Param(atype(prior_weights["pos_emb"][:cpu]()[:numpy]()))

model.state_emb.w = Param(atype(prior_weights["state_emb.weight"][:cpu]()[:numpy]()))
model.state_emb.b = Param(atype(prior_weights["state_emb.bias"][:cpu]()[:numpy]()))

for i in 1:model_config["n_layer"]
    model.blocks.layers[i].ln1.a = Param(atype(prior_weights["blocks.$(i-1).ln1.weight"][:cpu]()[:numpy]()))
    model.blocks.layers[i].ln1.b = Param(atype(prior_weights["blocks.$(i-1).ln1.bias"][:cpu]()[:numpy]()))
    model.blocks.layers[i].ln2.a = Param(atype(prior_weights["blocks.$(i-1).ln2.weight"][:cpu]()[:numpy]()))
    model.blocks.layers[i].ln2.b = Param(atype(prior_weights["blocks.$(i-1).ln2.bias"][:cpu]()[:numpy]()))

    model.blocks.layers[i].attn.key.w = Param(atype(prior_weights["blocks.$(i-1).attn.key.weight"][:cpu]()[:numpy]()))
    model.blocks.layers[i].attn.key.b = Param(atype(prior_weights["blocks.$(i-1).attn.key.bias"][:cpu]()[:numpy]()))
    model.blocks.layers[i].attn.query.w = Param(atype(prior_weights["blocks.$(i-1).attn.query.weight"][:cpu]()[:numpy]()))
    model.blocks.layers[i].attn.query.b = Param(atype(prior_weights["blocks.$(i-1).attn.query.bias"][:cpu]()[:numpy]()))
    model.blocks.layers[i].attn.value.w = Param(atype(prior_weights["blocks.$(i-1).attn.value.weight"][:cpu]()[:numpy]()))
    model.blocks.layers[i].attn.value.b = Param(atype(prior_weights["blocks.$(i-1).attn.value.bias"][:cpu]()[:numpy]()))
    model.blocks.layers[i].attn.proj.w = Param(atype(prior_weights["blocks.$(i-1).attn.proj.weight"][:cpu]()[:numpy]()))
    model.blocks.layers[i].attn.proj.b = Param(atype(prior_weights["blocks.$(i-1).attn.proj.bias"][:cpu]()[:numpy]()))

    model.blocks.layers[i].mlp.layers[1].w = Param(atype(prior_weights["blocks.$(i-1).mlp.0.weight"][:cpu]()[:numpy]()))
    model.blocks.layers[i].mlp.layers[1].b = Param(atype(prior_weights["blocks.$(i-1).mlp.0.bias"][:cpu]()[:numpy]()))
    model.blocks.layers[i].mlp.layers[3].w = Param(atype(prior_weights["blocks.$(i-1).mlp.2.weight"][:cpu]()[:numpy]()))
    model.blocks.layers[i].mlp.layers[3].b = Param(atype(prior_weights["blocks.$(i-1).mlp.2.bias"][:cpu]()[:numpy]()))
end

model.ln_f.a = Param(atype(prior_weights["ln_f.weight"][:cpu]()[:numpy]()))
model.ln_f.b = Param(atype(prior_weights["ln_f.bias"][:cpu]()[:numpy]()))

model.head.w = Param(atype(prior_weights["head.weight"][:cpu]()[:numpy]())); # no bias

# Test same results